# Bài thực hành số 5 lớp (DS108.P21.2)

## Bài 1 

In [1]:
# Bài 1: Đọc dữ liệu
# Đọc và load các file dữ liệu từ tập UCI HAR Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Đường dẫn đến dataset
data_path = 'getdata_projectfiles_UCI HAR Dataset/UCI HAR Dataset/'

# Đọc metadata
features = pd.read_csv(data_path + 'features.txt', sep=' ', header=None, names=['id', 'feature'])
activity_labels = pd.read_csv(data_path + 'activity_labels.txt', sep=' ', header=None, names=['id', 'activity'])

# Đọc dữ liệu train
X_train = pd.read_csv(data_path + 'train/X_train.txt', sep='\s+', header=None)
y_train = pd.read_csv(data_path + 'train/y_train.txt', header=None, names=['activity'])
subject_train = pd.read_csv(data_path + 'train/subject_train.txt', header=None, names=['subject'])

# Đọc dữ liệu test
X_test = pd.read_csv(data_path + 'test/X_test.txt', sep='\s+', header=None)
y_test = pd.read_csv(data_path + 'test/y_test.txt', header=None, names=['activity'])
subject_test = pd.read_csv(data_path + 'test/subject_test.txt', header=None, names=['subject'])

# In thông tin kích thước dữ liệu
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:20: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:20: SyntaxWarning: invalid escape sequence '\s'
C:\Users\APTS\AppData\Local\Temp\ipykernel_19048\3663590884.py:15: SyntaxWarning: invalid escape sequence '\s'
  X_train = pd.read_csv(data_path + 'train/X_train.txt', sep='\s+', header=None)
C:\Users\APTS\AppData\Local\Temp\ipykernel_19048\3663590884.py:20: SyntaxWarning: invalid escape sequence '\s'
  X_test = pd.read_csv(data_path + 'test/X_test.txt', sep='\s+', header=None)


X_train shape: (7352, 561)
y_train shape: (7352, 1)
X_test shape: (2947, 561)
y_test shape: (2947, 1)


## Bài 2


In [2]:
# Bài 2: Xử lý dữ liệu

# Đặt tên cột cho dữ liệu X theo features.txt
X_train.columns = features['feature']
X_test.columns = features['feature']

# Ghép dữ liệu train và test
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])
subjects = pd.concat([subject_train, subject_test])

# Thêm tên hoạt động vào y
y['activity_name'] = y['activity'].map(activity_labels.set_index('id')['activity'])

# Hiển thị thông tin cơ bản về dữ liệu
print("\nThông tin về dữ liệu:")
print("Tổng số mẫu:", X.shape[0])
print("Số lượng đặc trưng:", X.shape[1])
print("Phân bố các hoạt động:\n", y['activity_name'].value_counts())


Thông tin về dữ liệu:
Tổng số mẫu: 10299
Số lượng đặc trưng: 561
Phân bố các hoạt động:
 activity_name
LAYING                1944
STANDING              1906
SITTING               1777
WALKING               1722
WALKING_UPSTAIRS      1544
WALKING_DOWNSTAIRS    1406
Name: count, dtype: int64


## Bài 3

In [3]:
# Bài 3: Phân tích dữ liệu thô
# Trực quan hóa phân bố các hoạt động
plt.figure(figsize=(12, 6))
sns.countplot(y=y['activity_name'])
plt.title('Phân bố các loại hoạt động')
plt.xlabel('Số lượng mẫu')
plt.ylabel('Hoạt động')
plt.tight_layout()
plt.savefig('activity_distribution.png')
plt.close()

# Phân tích phân bố đối tượng tham gia
plt.figure(figsize=(10, 6))
sns.countplot(x='subject', data=subjects)
plt.title('Số lượng mẫu theo đối tượng tham gia')
plt.xlabel('ID đối tượng')
plt.ylabel('Số lượng mẫu')
plt.tight_layout()
plt.savefig('subject_distribution.png')
plt.close()

# Chọn một số đặc trưng để trực quan hóa
selected_features_vis = ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 
                         'tBodyGyro-mean()-X', 'tBodyGyro-mean()-Y', 'tBodyGyro-mean()-Z']

# Tạo dữ liệu để vẽ boxplot theo từng hoạt động
data_for_vis = X[selected_features_vis].copy()
data_for_vis['activity'] = y['activity_name'].values

# Vẽ boxplot cho các đặc trưng gia tốc
plt.figure(figsize=(15, 10))
for i, feature in enumerate(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z']):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='activity', y=feature, data=data_for_vis)
    plt.xticks(rotation=90)
    plt.title(f'Boxplot của {feature}')

# Vẽ boxplot cho các đặc trưng góc quay
for i, feature in enumerate(['tBodyGyro-mean()-X', 'tBodyGyro-mean()-Y', 'tBodyGyro-mean()-Z']):
    plt.subplot(2, 3, i+4)
    sns.boxplot(x='activity', y=feature, data=data_for_vis)
    plt.xticks(rotation=90)
    plt.title(f'Boxplot của {feature}')

plt.tight_layout()
plt.savefig('features_boxplots.png')
plt.close()

## Bài 4

In [4]:
# Bài 4: Chuẩn hóa dữ liệu
from sklearn.preprocessing import StandardScaler

# Chỉ chọn các đặc trưng có chứa mean() và std()
mean_std_features = [col for col in X.columns if 'mean()' in col or 'std()' in col]
X_selected = X[mean_std_features]

print(f"Số đặc trưng sau khi chọn (chỉ mean và std): {X_selected.shape[1]}")

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# Chuyển lại về DataFrame để dễ thao tác
X_scaled_df = pd.DataFrame(X_scaled, columns=mean_std_features)

# Hiển thị thống kê sau khi chuẩn hóa
print("\nThống kê sau khi chuẩn hóa:")
print(X_scaled_df.describe().round(2).T[['mean', 'std', 'min', 'max']])

# Trực quan hóa một số đặc trưng sau chuẩn hóa
plt.figure(figsize=(12, 6))
for i, feature in enumerate(mean_std_features[:5]):  # Chỉ hiển thị 5 đặc trưng đầu tiên
    plt.subplot(1, 5, i+1)
    sns.histplot(X_scaled_df[feature], kde=True)
    plt.title(f'{feature}')
    plt.xlabel('Giá trị')
    plt.ylabel('Tần suất')

plt.tight_layout()
plt.savefig('normalized_features.png')
plt.close()

Số đặc trưng sau khi chọn (chỉ mean và std): 66

Thống kê sau khi chuẩn hóa:
                             mean  std    min    max
tBodyAcc-mean()-X            -0.0  1.0 -18.84  10.73
tBodyAcc-mean()-Y             0.0  1.0 -26.46  27.41
tBodyAcc-mean()-Z            -0.0  1.0 -16.80  20.91
tBodyAcc-std()-X             -0.0  1.0  -0.89   3.67
tBodyAcc-std()-Y             -0.0  1.0  -0.98   3.02
...                           ...  ...    ...    ...
fBodyBodyAccJerkMag-std()     0.0  1.0  -0.89   4.04
fBodyBodyGyroMag-mean()       0.0  1.0  -0.93   5.24
fBodyBodyGyroMag-std()       -0.0  1.0  -0.97   5.48
fBodyBodyGyroJerkMag-mean()   0.0  1.0  -0.82   6.65
fBodyBodyGyroJerkMag-std()   -0.0  1.0  -0.80   6.92

[66 rows x 4 columns]


## Bài 5

In [5]:
# Bài 5: Giảm chiều dữ liệu với PCA
from sklearn.decomposition import PCA

# Áp dụng PCA để giữ lại 95% phương sai
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

print(f"Số chiều sau khi áp dụng PCA: {X_pca.shape[1]}")
print(f"Tổng phương sai giải thích: {sum(pca.explained_variance_ratio_)*100:.2f}%")

# Trực quan hóa phương sai giải thích
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_)
plt.plot(range(1, len(pca.explained_variance_ratio_)+1), np.cumsum(pca.explained_variance_ratio_), 'r-')
plt.axhline(y=0.95, color='g', linestyle='--')
plt.title('Phương sai giải thích của các thành phần chính')
plt.xlabel('Thành phần chính')
plt.ylabel('Tỷ lệ phương sai giải thích')
plt.savefig('pca_variance.png')
plt.close()

# Trực quan hóa dữ liệu sau khi giảm chiều xuống 2D
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

plt.figure(figsize=(12, 10))
scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y['activity'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Activity ID')
plt.title('Dữ liệu sau khi giảm chiều xuống 2D với PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.savefig('pca_2d_visualization.png')
plt.close()

Số chiều sau khi áp dụng PCA: 17
Tổng phương sai giải thích: 95.29%


## Bài 6

In [6]:
# Bài 6: Huấn luyện mô hình và đánh giá
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train_pca, X_test_pca, y_train_labels, y_test_labels = train_test_split(
    X_pca, y['activity'], test_size=0.3, random_state=42, stratify=y['activity']
)

# Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_pca, y_train_labels)

# Dự đoán trên tập kiểm tra
y_pred = rf_model.predict(X_test_pca)

# Đánh giá mô hình
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"\nĐộ chính xác của mô hình: {accuracy:.4f}")

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test_labels, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=activity_labels['activity'],
            yticklabels=activity_labels['activity'])
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title('Ma trận nhầm lẫn')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.close()

# Báo cáo phân loại chi tiết
print("\nBáo cáo phân loại chi tiết:")
print(classification_report(y_test_labels, y_pred, target_names=activity_labels['activity']))


Độ chính xác của mô hình: 0.8951

Báo cáo phân loại chi tiết:
                    precision    recall  f1-score   support

           WALKING       0.90      0.86      0.88       517
  WALKING_UPSTAIRS       0.83      0.92      0.87       463
WALKING_DOWNSTAIRS       0.87      0.82      0.85       422
           SITTING       0.87      0.87      0.87       533
          STANDING       0.89      0.88      0.89       572
            LAYING       1.00      0.99      0.99       583

          accuracy                           0.90      3090
         macro avg       0.89      0.89      0.89      3090
      weighted avg       0.90      0.90      0.90      3090



## Bài 7

In [7]:
# Bài 7: So sánh các mô hình
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Định nghĩa các mô hình để so sánh
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', C=1, gamma='scale', random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
}

# Dictionary để lưu kết quả
results = {}

# Huấn luyện và đánh giá từng mô hình
for name, model in models.items():
    print(f"\nHuấn luyện mô hình {name}...")
    model.fit(X_train_pca, y_train_labels)
    
    # Dự đoán và tính độ chính xác
    y_pred = model.predict(X_test_pca)
    accuracy = accuracy_score(y_test_labels, y_pred)
    results[name] = accuracy
    
    print(f"Độ chính xác của {name}: {accuracy:.4f}")

# Trực quan hóa kết quả so sánh các mô hình
plt.figure(figsize=(10, 6))
sns.barplot(x=list(results.keys()), y=list(results.values()))
plt.title('So sánh độ chính xác của các mô hình')
plt.xlabel('Mô hình')
plt.ylabel('Độ chính xác')
plt.ylim(0.8, 1.0)  # Điều chỉnh để thấy rõ sự khác biệt
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('model_comparison.png')
plt.close()


Huấn luyện mô hình Random Forest...
Độ chính xác của Random Forest: 0.8951

Huấn luyện mô hình SVM...
Độ chính xác của SVM: 0.8916

Huấn luyện mô hình KNN...
Độ chính xác của KNN: 0.8848

Huấn luyện mô hình Neural Network...


d:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Độ chính xác của Neural Network: 0.9126


## Bài 8

In [8]:
# Bài 8: Phân tích đặc trưng quan trọng
# Lấy mô hình Random Forest để phân tích đặc trưng quan trọng
rf_model = models['Random Forest']
feature_importances = rf_model.feature_importances_

# Tạo DataFrame để dễ dàng thao tác
if X_pca.shape[1] <= 30:  # Nếu số đặc trưng sau PCA nhỏ, sử dụng trực tiếp
    importances_df = pd.DataFrame({
        'feature': [f'PC{i+1}' for i in range(X_pca.shape[1])],
        'importance': feature_importances
    })
    
    # Sắp xếp theo tầm quan trọng giảm dần
    importances_df = importances_df.sort_values('importance', ascending=False)
    
    # Trực quan hóa
    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=importances_df.head(15))
    plt.title('Top 15 thành phần PCA quan trọng nhất')
    plt.tight_layout()
    plt.savefig('feature_importance_pca.png')
    plt.close()
else:
    # Nếu số lượng thành phần PCA lớn, chỉ hiển thị top 20
    importances = [(i, f'PC{i+1}', imp) for i, imp in enumerate(feature_importances)]
    importances.sort(key=lambda x: x[2], reverse=True)
    
    top_components = importances[:20]
    
    # Vẽ biểu đồ 
    plt.figure(figsize=(12, 8))
    plt.bar([comp[1] for comp in top_components], [comp[2] for comp in top_components])
    plt.title('Top 20 thành phần PCA quan trọng nhất')
    plt.xlabel('Thành phần PCA')
    plt.ylabel('Tầm quan trọng')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig('feature_importance_pca_top20.png')
    plt.close()
    
    # Phân tích thành phần chính đóng góp gì vào đặc trưng gốc
    top_pca_indices = [comp[0] for comp in top_components[:5]]  # Lấy 5 thành phần quan trọng nhất
    
    # Phân tích 5 thành phần quan trọng nhất
    plt.figure(figsize=(15, 12))
    
    for i, idx in enumerate(top_pca_indices):
        # Lấy trọng số đóng góp của các đặc trưng gốc vào thành phần này
        component = pd.Series(abs(pca.components_[idx]), index=mean_std_features)
        component = component.sort_values(ascending=False)
        
        # Vẽ biểu đồ top 10 đặc trưng gốc đóng góp nhiều nhất
        plt.subplot(3, 2, i+1)
        sns.barplot(x=component.values[:10], y=component.index[:10])
        plt.title(f'Top 10 đặc trưng đóng góp cho PC{idx+1}')
        
    plt.tight_layout()
    plt.savefig('pca_components_analysis.png')
    plt.close()

print("\nPhân tích hoàn tất! Các biểu đồ đã được lưu.")


Phân tích hoàn tất! Các biểu đồ đã được lưu.


## Bài 9

In [9]:
# Bài 9: Tổng kết và báo cáo kết quả
# Hiển thị tóm tắt về quá trình và kết quả

print("\n===== TÓM TẮT KẾT QUẢ =====")
print(f"Số lượng mẫu ban đầu: {X.shape[0]}")
print(f"Số lượng đặc trưng ban đầu: {X.shape[1]}")
print(f"Số lượng đặc trưng sau khi chọn (mean và std): {X_selected.shape[1]}")
print(f"Số lượng thành phần sau PCA (95% phương sai): {X_pca.shape[1]}")
print("\nKết quả độ chính xác của các mô hình:")
for name, acc in results.items():
    print(f"- {name}: {acc:.4f}")

# Mô hình tốt nhất
best_model = max(results, key=results.get)
print(f"\nMô hình tốt nhất: {best_model} với độ chính xác {results[best_model]:.4f}")

# Kết luận
print("\nKết luận:")
print("1. Dữ liệu UCI HAR Dataset chứa các đặc trưng về gia tốc và góc quay từ smartphone.")
print("2. Đã giảm chiều dữ liệu từ nhiều đặc trưng xuống còn ít hơn nhưng vẫn giữ được 95% thông tin.")
print(f"3. {best_model} cho kết quả tốt nhất trong việc phân loại 6 hoạt động khác nhau.")
print("4. Các đặc trưng quan trọng đã được phân tích và trực quan hóa.")
print("5. Kết quả cho thấy mô hình có thể nhận dạng hoạt động con người với độ chính xác cao.")


===== TÓM TẮT KẾT QUẢ =====
Số lượng mẫu ban đầu: 10299
Số lượng đặc trưng ban đầu: 561
Số lượng đặc trưng sau khi chọn (mean và std): 66
Số lượng thành phần sau PCA (95% phương sai): 17

Kết quả độ chính xác của các mô hình:
- Random Forest: 0.8951
- SVM: 0.8916
- KNN: 0.8848
- Neural Network: 0.9126

Mô hình tốt nhất: Neural Network với độ chính xác 0.9126

Kết luận:
1. Dữ liệu UCI HAR Dataset chứa các đặc trưng về gia tốc và góc quay từ smartphone.
2. Đã giảm chiều dữ liệu từ nhiều đặc trưng xuống còn ít hơn nhưng vẫn giữ được 95% thông tin.
3. Neural Network cho kết quả tốt nhất trong việc phân loại 6 hoạt động khác nhau.
4. Các đặc trưng quan trọng đã được phân tích và trực quan hóa.
5. Kết quả cho thấy mô hình có thể nhận dạng hoạt động con người với độ chính xác cao.
